In [ ]:
!pip install cvxpy==1.1.17

In [ ]:
!pip install -U -q scipy

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
import cvxpy as cp
import numpy as np
import scipy as scipy
from scipy.sparse import random
from scipy.linalg import toeplitz
from numpy import linalg as la
from scipy.linalg import sqrtm
from numpy.ma.core import size
from scipy import io
import matplotlib.pyplot as plt
import h5py
from google.colab import files



In [ ]:
n = 32                                                         # initialize number of nodes
def GLASSO_33bus(N,alpha,t):                                   #define the GLASSO+SR estimator with N samples, alpha is the regularization parameter and t trials
  trials =np.arange(1,t)
  Sc = []
  Ss = []
  Er = []
  count = 0
  B = scipy.io.loadmat('/content/drive/My Drive/33bus_modified_ybus.mat')      #load the power distribution network data-set
  B = B['xx']
  B = np.array(B)
  B = np.delete(B,0,0) #delete 1st row
  B = np.delete(B,0,1) #delete 1st column
  B[24,26] = B[26,24] = B[10,12] = B[12,10] = B[6,9] = B[9,6] = B[26,29] = B[29,26] = B[21,3] = B[3,21] = B[5,7] = B[7,5] = 1      #add 3 cycles of length 3, 2 cycles of length 4 and one cycle of length 5
  B = np.where(B !=0,1,0)
  B = B + 2*np.mat(np.eye(32))                               # weighting the diagonals to ensure positive definiteness    
  for i in range(len(trials)):                               
    Binv = np.linalg.inv(B)
    Theta_X = np.matrix(np.eye(n))
    Theta_Y = Binv*Binv
    Sigma_X = np.linalg.inv(Theta_X)
    Sigma_Y = np.linalg.inv(Theta_Y)
    y_sample = scipy.linalg.sqrtm(Sigma_Y) * np.matrix(np.random.randn(n, N))
    S = np.cov(y_sample)                                    #construct sample covariance matrix
    S = np.real(S)
    Sc.append(S)
  for i in range(len(trials)):
    Theta_hat = cp.Variable(shape=(n,n), PSD = True)
    obj = cp.Minimize( -cp.log_det(Theta_hat) + cp.trace(Sc[i]@Theta_hat) + alpha*cp.sum(cp.abs(Theta_hat)))         #Set-up the convex optimization problem
    prob = cp.Problem(obj)
    prob.solve()
    if prob.status != cp.OPTIMAL:
      raise Exception('CVXPY Error')
    Theta_hat = Theta_hat.value
    Theta_hat = scipy.linalg.sqrtm(Theta_hat)                 # Taking square root of GLASSO estimate
    Theta_hat[abs(Theta_hat) <= 1e-2] = 0                     #Thresholding entries of the estimator
    Error = np.linalg.norm(Theta_hat - B,1)
    Ss.append(Theta_hat)
    Er += [Error]
    print('Completed optimization parameterized by alpha = {}, obj value = {}, Inf norm = {} '.format(alpha, obj.value,Error))
    B_opt = np.where(Theta_hat!=0,1,0)
    B = np.where(B!=0,1,0)
    if np.array_equal(B_opt,B) == True:
      count+=1
  plt.spy(Theta_hat)
  return count

def QGLASSO_33bus(N,alpha,t):                   #define the ML estimator with N samples, alpha is the regularization parameter and t trials
  trials =np.arange(1,t)
  Dd = []
  Sc = []
  Vv = []
  Ss = []
  Er = []
  count = 0
  B = scipy.io.loadmat('/content/drive/My Drive/33bus_modified_ybus.mat')       #load the power distribution network data-set
  B = B['xx']
  B = np.array(B)
  B = np.delete(B,0,0) #delete 1st row
  B = np.delete(B,0,1) #delete 1st column
  B[24,26] = B[26,24] = B[10,12] = B[12,10] = B[6,9] = B[9,6] = B[26,29] = B[29,26] = B[21,3] = B[3,21] = B[5,7] = B[7,5] = 1           #add 3 cycles of length 3, 2 cycles of length 4 and one cycle of length 5
  B = np.where(B !=0,1,0)
  B = B + 2*np.mat(np.eye(32))                          # weighting the diagonals to ensure positive definiteness 
  evalue,evec = la.eigh(B)
  print(evalue)
  
  for i in range(len(trials)):
    Binv = np.linalg.inv(B)
    Theta_X = np.matrix(np.eye(n))
    Theta_Y = Binv*Binv
    Sigma_X = np.linalg.inv(Theta_X)
    Sigma_Y = np.linalg.inv(Theta_Y)
    y_sample = scipy.linalg.sqrtm(Sigma_Y) * np.matrix(np.random.randn(n, N))
    S = np.cov(y_sample)                                #construct sample covariance matrix
    Sc.append(S)
  for i in range(len(trials)):
    evalue,evec = la.eigh(Sc[i])
    Lambda = np.diag(evalue)
    R = scipy.linalg.sqrtm(Lambda)
    V = evec@R
    Vv.append(V)
  for i in range(len(trials)):
    Theta_hat = cp.Variable(shape=(n,n), PSD=True)
    obj = cp.Minimize(-2*cp.log_det(Theta_hat) +  (cp.norm(Theta_hat@Vv[i], "fro"))**2 + alpha*cp.sum(cp.abs(Theta_hat)))           #Set-up the convex optimization problem
    prob = cp.Problem(obj)
    prob.solve()
    if prob.status != cp.OPTIMAL:
          raise Exception('CVXPY Error')
    Theta_hat = Theta_hat.value
    Theta_hat[abs(Theta_hat) <= 1e-2] = 0       #Thresholding entries of the estimator
    Error = np.linalg.norm( Theta_hat - B,1)
    Ss.append(Theta_hat)
    Er += [Error]
    print('Completed optimization parameterized by alpha = {}, obj value = {}, Inf norm = {} '.format(alpha, obj.value,Error))
    B_opt = np.where(Theta_hat!=0,1,0)
    B = np.where(B!=0,1,0)
    if np.array_equal(B_opt,B) == True:
      count+=1
  plt.spy(Theta_hat)
  return count

def Deka(N,alpha,t):                          #define the GLASSO+2HR estimator with N samples, alpha is the regularization parameter and t trials
  trials =np.arange(1,t)
  Sc = []
  Ss = []
  Er = []
  count = 0
  B = scipy.io.loadmat('/content/drive/My Drive/33bus_modified_ybus.mat')         #load the power distribution network data-set
  B = B['xx']
  B = B['xx']
  B = np.array(B)
  B = np.delete(B,0,0) #delete 1st row
  B = np.delete(B,0,1) #delete 1st column
  B[24,26] = B[26,24] = B[10,12] = B[12,10] = B[6,9] = B[9,6] = B[26,29] = B[29,26] = B[21,3] = B[3,21] = B[5,7] = B[7,5] = 1         #add 3 cycles of length 3, 2 cycles of length 4 and one cycle of length 5
  B = np.where(B !=0,1,0)
  B = B + 2*np.mat(np.eye(32))            # weighting the diagonals to ensure positive definiteness
  for i in range(len(trials)):
    Binv = np.linalg.inv(B)
    Theta_X = np.matrix(np.eye(n))
    Theta_Y = Binv*Binv
    Sigma_X = np.linalg.inv(Theta_X)
    Sigma_Y = np.linalg.inv(Theta_Y)
    y_sample = scipy.linalg.sqrtm(Sigma_Y) * np.matrix(np.random.randn(n, N))
    S = np.cov(y_sample)                  #construct sample covariance matrix
    S = np.real(S)
    Sc.append(S)
  for i in range(len(trials)):
    Theta_hat = cp.Variable(shape=(n,n), PSD = True)
    obj = cp.Minimize( -cp.log_det(Theta_hat) + cp.trace(Sc[i]@Theta_hat) + alpha*cp.sum(cp.abs(Theta_hat)))
    prob = cp.Problem(obj)
    prob.solve()
    if prob.status != cp.OPTIMAL:
      raise Exception('CVXPY Error')
    Theta_hat = Theta_hat.value
    for i in range(n):
      Theta_hat[i,i] = -1e-2
    Theta_hat[Theta_hat <= -1e-2] = 1      #Thresholding negative entries of the estimator
    Theta_hat = np.where(Theta_hat !=1,0,1)
    Error = np.linalg.norm(Theta_hat - B,1)
    Ss.append(Theta_hat)
    Er += [Error]
    print('Completed optimization parameterized by alpha = {}, obj value = {}, Inf norm = {} '.format(alpha, obj.value,Error))
    B = np.where(B!=0,1,0)
    if np.array_equal(Theta_hat,B) == True:
      count+=1
  plt.spy(Theta_hat)
  return count


In [ ]:
prob_gsuccess = []                                #Initiliazing the probability of success for GLASSO+SR estimator for power distribution network with 32 nodes
samples = np.arange(200,1800,150)
for s in samples:
  success = GLASSO_33bus(s,0.8,101)               # Running GLASSO+SR estimator on 32 node power distribution network for 100 trials
  prob_gsuccess.append(success)
prob_gsuccess = np.array(prob_gsuccess)
prob_gsuccess = prob_gsuccess/100
print(prob_gsuccess)

In [ ]:
prob_success = []                                 #Initiliazing the probability of success for ML estimator for power distribution network with 32 nodes
samples = np.arange(200,1800,150)
for s in samples:
  success = QGLASSO_33bus(s,0.8,101)              # Running ML estimator on 32 node power distribution network for 100 trials
  prob_success.append(success)
prob_success = np.array(prob_success)
prob_success = prob_success/100
print(prob_success)

In [ ]:
prob_dsuccess = []                                #Initiliazing the probability of success for GLASSO+2HR estimator for power distribution network with 32 nodes
samples = np.arange(200,1800,150)
for s in samples:
  success = Deka(s,0.27,101)                      # Running ML estimator on 32 node power distribution network for 100 trials
  prob_dsuccess.append(success)
prob_dsuccess = np.array(prob_dsuccess)
prob_dsuccess = prob_dsuccess/100
print(prob_dsuccess)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mt
from google.colab import files
plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = ['Times New Roman']
fig = plt.figure()
plt.rc('xtick', labelsize = 14)
plt.rc('ytick', labelsize = 14)
fig.set_size_inches(6, 6)
plt.rc('axes', labelsize=21)
samples = np.arange(200,1800,150)
prob_gsuccess = np.array([0,0.01,0.12,0.27,0.48,0.47,0.67,0.68,0.74,0.81,0.86])
prob_success = np.array([0.05,0.49,0.65,0.83,0.84,0.95,0.97,0.99,1,1,1])
prob_dsuccess = np.array([0,0.01,0.28,0.45,0.62,0.67,0.69,0.83,0.82,0.89,0.88])
plt.plot(samples,prob_success,'-rv',label = 'MLE')
plt.plot(samples, prob_dsuccess,'--g^',label = 'GL+2HR')
plt.plot(samples,prob_gsuccess,'-bs', label = 'GLASSO+SR')
plt.xlabel(r'number of samples (n)')
plt.ylabel('Probability of support recovery')
plt.title('IEEE 33 bus power distribution network')
plt.xlim(0, 1800)
plt.ylim(0)
plt.grid(True, linestyle = '--')
plt.legend()
plt.savefig("IEEE1", bbox_inches='tight')
plt.savefig("IEEE1.pdf")
plt.show()

# New section